In [ ]:
# -*- coding: utf-8 -*-
"""Capstone_Project.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1tzm7K11OgybQAa1MeKSG4FNlU6T0YNvy

# Imports and Preprocessing
"""

import numpy as np
import os
import PIL
import PIL.Image
from PIL import Image, ImageOps
import tensorflow as tf
import tensorflow_datasets as tfds
import glob
import pathlib
from google.colab import files
from google.colab import drive
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Dropout, Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.applications import VGG16
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle as pkl
from IPython.display import clear_output
from time import sleep
import sys
import requests
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import warnings
import pickle
import gc
from statistics import median

warnings.filterwarnings('ignore')

!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install
clear_output()

print(tf.__version__)

drive.mount("/content/drive/")

!ls "/content/drive/My Drive/Capstone/anime/animated2"

"""# Anger DataFrame"""

anger = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_anger/*.png')
anger_names = [str(i)[::-1].split("/", 1)[0][::-1] for i in anger]
anger_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in anger]
cartoon_anger = pd.DataFrame()
cartoon_anger["Directories"], cartoon_anger["Names"], cartoon_anger["Emotion"], cartoon_anger["Y"] = anger_directories, anger_names, "anger", 0

cartoon_anger

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_anger/aia_anger_1.png")
plt.imshow(img)

"""# Joy DataFrame"""

joy = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_joy/*.png')
joy_names = [str(i)[::-1].split("/", 1)[0][::-1] for i in joy]
joy_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in joy]
cartoon_joy = pd.DataFrame()
cartoon_joy["Directories"], cartoon_joy["Names"], cartoon_joy["Emotion"], cartoon_joy["Y"] = joy_directories, joy_names, "joy", 1

cartoon_joy

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_joy/jules_joy_1.png")
plt.imshow(img)

"""# Surprise DataFrame"""

surprise = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_surprise/*.png')
surprise_names = [str(i)[::-1].split("/", 1)[0][::-1] for i in surprise]
surprise_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in surprise]
cartoon_surprise = pd.DataFrame()
cartoon_surprise["Directories"], cartoon_surprise["Names"], cartoon_surprise["Emotion"], cartoon_surprise["Y"] = surprise_directories, surprise_names, "surprise", 2

cartoon_surprise

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_surprise/malcolm_surprise_1.png")
plt.imshow(img)

"""# Disgust DataFrame"""

disgust = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_disgust/*.png')
disgust_names = [str(i)[::-1].split("/", 1)[0][::-1] for i in disgust]
disgust_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in disgust]
cartoon_disgust = pd.DataFrame()
cartoon_disgust["Directories"], cartoon_disgust["Names"], cartoon_disgust["Emotion"], cartoon_disgust["Y"] = disgust_directories, disgust_names, "disgust", 3

cartoon_disgust

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_disgust/bonnie_disgust_1.png")
plt.imshow(img)

"""# Fear DataFrame"""

fear = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_fear/*.png')
fear_names= [str(i)[::-1].split("/", 1)[0][::-1] for i in fear]
fear_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in fear]
cartoon_fear = pd.DataFrame()
cartoon_fear["Directories"], cartoon_fear["Names"], cartoon_fear["Emotion"], cartoon_fear["Y"] = fear_directories, fear_names, "fear", 4

cartoon_fear

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_fear/mery_fear_1.png")
plt.imshow(img)

"""# Sadness DataFrame"""

sadness = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_sadness/*.png')
sadness_names = [str(i)[::-1].split("/", 1)[0][::-1] for i in sadness]
sadness_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in sadness]
cartoon_sadness = pd.DataFrame()
cartoon_sadness["Directories"], cartoon_sadness["Names"], cartoon_sadness["Emotion"], cartoon_sadness["Y"] = sadness_directories, sadness_names, "sadness", 5

cartoon_sadness

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_sadness/ray_sadness_1.png")
plt.imshow(img)

"""# Neutral DataFrame"""

neutral = glob.glob('/content/drive/MyDrive/Capstone/anime/animated2/cartoon_neutral/*.png')
neutral_names = [str(i)[::-1].split("/", 1)[0][::-1] for i in neutral]
neutral_directories = [str(i)[::-1].split("/", 1)[1][::-1] for i in neutral]
cartoon_neutral = pd.DataFrame()
cartoon_neutral["Directories"], cartoon_neutral["Names"], cartoon_neutral["Emotion"], cartoon_neutral["Y"] = neutral_directories, neutral_names, "neutral", 6

cartoon_neutral

img = cv2.imread("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_neutral/jules_neutral_1.png")
plt.imshow(img)

"""# Concatenating all dataframes -> Cartoon DataFrame"""

cartoon = pd.concat([cartoon_anger, cartoon_joy, cartoon_surprise, cartoon_disgust, cartoon_fear, cartoon_sadness, cartoon_neutral])

cartoon

#Train test split
cartoon_train, cartoon_test = train_test_split(cartoon, stratify=cartoon["Y"], test_size = 0.2)

cartoon_train

"""# Loading Saved DataFrames"""

# cartoon_train.to_csv("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_train.csv")
# cartoon_test.to_csv("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_test.csv")

cartoon_train = pd.read_csv("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_train.csv")
cartoon_test = pd.read_csv("/content/drive/MyDrive/Capstone/anime/animated2/cartoon_test.csv")

cartoon_train["Y"].value_counts()

df = cartoon_train.sample(frac = 1)
df = df.reset_index()
df

"""# Model Configuration and Training"""

def get_models():
  model_bottleneck = VGG16(weights='imagenet', include_top=False)
  model_cartoon = load_model("/content/drive/MyDrive/Capstone/anime/animated2/ResizedModelSingleEpoch.h5")
  model_cartoon.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])
  
  return model_bottleneck, model_cartoon

# Transfer learning -> Using pretrained VGG with imagenet weights. Fine tuning to be done with cartoon images

batch_size = 20 # Stream and train batches of 20 images at once. Cannot load entire dataset into limited RAM
ishape = 8*8*512 # Shape of VGG output

# Train model for 1 epoch. Time taken ~ 3 hours

modelvgg = Sequential()
modelvgg.add(Flatten())
modelvgg.add(Dense(512, activation='relu', input_shape = (ishape, )))
modelvgg.add(Dropout(0.2))
modelvgg.add(Dense(256, activation='relu'))
modelvgg.add(Dense(128, activation='relu'))
modelvgg.add(BatchNormalization())
modelvgg.add(Dense(64, activation='relu'))
modelvgg.add(Dense(7, activation='softmax')) 
modelvgg.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])

# for i in range(int(len(df)/batch_size)):
#   try:
#     print("Training on batch " +str(i+1))
#     temp_df = df[i*batch_size:(i+1)*batch_size]
#     images, labels = get_cartoon_images_and_labels(temp_df)
#     images = convert_cartoon_grayscale(resize_cartoon_images(images))
#     predictions = model.predict(images)
#     modelvgg.train_on_batch(np.asarray(predictions), labels)
#     del images
#     del labels
#     del temp_df
#   except:
#     print("Error on batch " + str(i+1))

# Accuracy here is 100%, but our end goal is to obtain a better accuracy on human images.
# modelvgg.save("/content/drive/MyDrive/Capstone/anime/animated2/ResizedModelSingleEpoch.h5")
# Load saved model from drive and test accuracy = 100%

"""# Image preprocessing"""

def get_human_paths():
  human_anger_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/CK+48/anger/*.png")
  df0 = pd.DataFrame({'Path':human_anger_paths, 'Label':[0 for i in human_anger_paths]})
  human_disgust_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/CK+48/disgust/*.png")
  df3 = pd.DataFrame({'Path':human_disgust_paths, 'Label':[3 for i in human_disgust_paths]})
  human_fear_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/CK+48/fear/*.png")
  df4 = pd.DataFrame({'Path':human_fear_paths, 'Label':[4 for i in human_fear_paths]})
  human_sadness_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/CK+48/sadness/*.png")
  df5 = pd.DataFrame({'Path':human_sadness_paths, 'Label':[5 for i in human_sadness_paths]})
  human_joy_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/CK+48/happy/*.png")
  df1 = pd.DataFrame({'Path':human_joy_paths, 'Label':[1 for i in human_joy_paths]})
  human_surprise_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/CK+48/surprise/*.png")
  df2 = pd.DataFrame({'Path':human_surprise_paths, 'Label':[2 for i in human_surprise_paths]})
  human_paths = human_anger_paths + human_disgust_paths + human_fear_paths + human_joy_paths + human_sadness_paths + human_surprise_paths
  human_df = pd.concat([df0, df1, df2, df3, df4, df5]).reset_index()

  return human_paths, human_df

def get_cartoon_images_and_labels(temp_df, integer_labels = False):
  paths = [i+"/"+j for i,j in zip(temp_df["Directories"].tolist(), temp_df["Names"].tolist())]
  images = np.asarray([np.asarray(Image.open(image))[:, :, :3] for image in paths])
  labels = np.asarray(temp_df["Y"].tolist())
  labels = to_categorical(labels, num_classes=7)
  integerLabels = np.asarray([np.argmax(label) for label in labels])

  if (integer_labels):
    return images, labels, integerLabels
  else:
    return images, labels

def resize_cartoon_images(images):
  clipped_images = [image[40:240, 35:225] for image in images]
  resized_clipped_images = [cv2.resize(clipped_image, (256, 256)) for clipped_image in clipped_images]
  return np.asarray(resized_clipped_images)

def convert_cartoon_grayscale(images):
  return np.asarray([np.asarray(ImageOps.grayscale(Image.fromarray(image)).convert("RGB")) for image in images])

def display_images(images):
  for image in images:
    display(Image.fromarray(image))
    print()

def get_human_cartoonized_images(human_ct_df, integer_labels = False, augment = False):
  paths = human_ct_df["Path"]
  if (augment):
    human_ct_images = np.concatenate((np.asarray([np.fliplr(np.asarray(ImageOps.grayscale(Image.open(path)).convert("RGB"))) for path in paths]), np.asarray([np.asarray(ImageOps.grayscale(Image.open(path)).convert("RGB")) for path in paths])), axis = 0)
    human_ct_labels = np.concatenate((np.asarray(human_ct_df["Label"].tolist()), np.asarray(human_ct_df["Label"].tolist())), axis = 0)
    human_ct_labels = np.asarray(to_categorical(human_ct_labels, num_classes=7))
  else:
    human_ct_images = np.asarray([np.asarray(ImageOps.grayscale(Image.open(path)).convert("RGB")) for path in paths])
    human_ct_labels = np.asarray(human_ct_df["Label"].tolist())
    human_ct_labels = np.asarray(to_categorical(human_ct_labels, num_classes=7))

  integerLabels = np.asarray([np.argmax(label) for label in human_ct_labels])

  if (integer_labels):
    return human_ct_images, human_ct_labels, integerLabels
  else:
    return human_ct_images, human_ct_labels

def get_human_images(human_df, integer_labels = False):
  paths = human_df["Path"]
  human_images = np.asarray([cv2.resize(np.asarray(Image.open(path).convert("RGB")), (256, 256)) for path in paths])
  human_labels = np.asarray(human_df["Label"].tolist())
  human_labels = np.asarray(to_categorical(human_labels, num_classes=7))
  integerLabels = np.asarray([np.argmax(label) for label in human_labels])

  if (integer_labels):
    return human_images, human_labels, integerLabels
  else:
    return human_images, human_labels

def train_human(model_bottleneck, model_cartoon, human_images, human_labels, epochs, lr = 0.1):
  human_bottleneck = np.asarray(model_bottleneck.predict(human_images))

  for epoch in range(epochs):
    #print("Training epoch " + str(epoch+1))
    for i in range(len(human_bottleneck)//20):
      model_cartoon.train_on_batch(human_bottleneck[20*i:20*(i+1)], human_labels[20*i:20*(i+1)])
  
  return model_cartoon

def get_human_cartoonized_paths():
  human_anger_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/shinkai/0*.png")
  df0 = pd.DataFrame({'Path':human_anger_paths, 'Label':[0 for i in human_anger_paths]})
  human_disgust_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/shinkai/3*.png")
  df3 = pd.DataFrame({'Path':human_disgust_paths, 'Label':[3 for i in human_disgust_paths]})
  human_fear_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/shinkai/4*.png")
  df4 = pd.DataFrame({'Path':human_fear_paths, 'Label':[4 for i in human_fear_paths]})
  human_sadness_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/shinkai/5*.png")
  df5 = pd.DataFrame({'Path':human_sadness_paths, 'Label':[5 for i in human_sadness_paths]})
  human_joy_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/shinkai/1*.png")
  df1 = pd.DataFrame({'Path':human_joy_paths, 'Label':[1 for i in human_joy_paths]})
  human_surprise_paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/shinkai/2*.png")
  df2 = pd.DataFrame({'Path':human_surprise_paths, 'Label':[2 for i in human_surprise_paths]})
  human_paths = human_anger_paths + human_disgust_paths + human_fear_paths + human_joy_paths + human_sadness_paths + human_surprise_paths
  human_df = pd.concat([df0, df1, df2, df3, df4, df5]).reset_index()

  return human_paths, human_df

"""# Accuracy Testing"""

def test_cartoon_accuracy(model_bottleneck, model_test, temp_df, display_image = True):
  temp_df = temp_df.reset_index()
  images, labels = get_cartoon_images_and_labels(temp_df)
  images = convert_cartoon_grayscale(resize_cartoon_images(images))
  bottleneck_features = model_bottleneck.predict(images)
  predictions = model_test.predict_classes(bottleneck_features)
  probabilities = model_test.predict(bottleneck_features)

  key = {0:"anger", 1:"joy", 2:"surprise", 3:"disgust", 4:"fear", 5:"sadness", 6:"neutral"}
  integerLabels = np.asarray([np.argmax(label) for label in labels])

  for index, image in list(enumerate(images)):
    if(display_image):
      display(Image.fromarray(image))
      print()
      print(temp_df["Names"][index])
      print("Predicted Class: " + str(key[predictions[index]]))
      print("Confidence in Prediction: " + str(np.max(probabilities[index])))
      print("Actual Class: " + str(key[integerLabels[index]]))
      print("__________________________________________________\n")

  accuracy = accuracy_score(integerLabels, predictions)
  metrics = precision_recall_fscore_support(integerLabels, predictions, average='weighted')
  confusion = confusion_matrix(integerLabels, predictions)

  return predictions, probabilities, accuracy, metrics, confusion

def test_human_accuracy(model_bottleneck, model_test, human_df, display_image = True):
  human_df = human_df.reset_index()
  human_images, human_labels = get_human_images(human_df)
  test_human_bottleneck =  np.asarray(model_bottleneck.predict(human_images))
  predictions = model_test.predict_classes(test_human_bottleneck)
  probabilities = model_test.predict(test_human_bottleneck)

  key = {0:"anger", 1:"joy", 2:"surprise", 3:"disgust", 4:"fear", 5:"sadness", 6:"neutral"}
  integerLabels = np.asarray([np.argmax(label) for label in human_labels])

  for index, image in list(enumerate(human_images)):
    if(display_image):
      display(Image.fromarray(image))
      print()
      print("Predicted Class: " + str(key[predictions[index]]))
      print("Confidence in Prediction: " + str(np.max(probabilities[index])))
      print("Actual Class: " + str(key[integerLabels[index]]))
      print("__________________________________________________\n")

  accuracy = accuracy_score(integerLabels, predictions)
  metrics = precision_recall_fscore_support(integerLabels, predictions, average='weighted')
  confusion = confusion_matrix(integerLabels, predictions)

  return predictions, probabilities, accuracy, metrics, confusion

def test_human_cartoonized_accuracy(model_bottleneck, model_test, human_ct_df, display_image = True, augment = False):
  human_ct_df = human_ct_df.reset_index()
  if (augment):
    human_ct_images, human_ct_labels = get_human_cartoonized_images(human_ct_df, augment = True)
  else:
    human_ct_images, human_ct_labels = get_human_cartoonized_images(human_ct_df)
  test_human_bottleneck =  np.asarray(model_bottleneck.predict(human_ct_images))
  predictions = model_test.predict_classes(test_human_bottleneck)
  probabilities = model_test.predict(test_human_bottleneck)

  key = {0:"anger", 1:"joy", 2:"surprise", 3:"disgust", 4:"fear", 5:"sadness", 6:"neutral"}
  integerLabels = np.asarray([np.argmax(label) for label in human_ct_labels])

  for index, image in list(enumerate(human_ct_images)):
    if(display_image):
      display(Image.fromarray(image))
      print()
      print("Predicted Class: " + str(key[predictions[index]]))
      print("Confidence in Prediction: " + str(np.max(probabilities[index])))
      print("Actual Class: " + str(key[integerLabels[index]]))
      print("__________________________________________________\n")

  accuracy = accuracy_score(integerLabels, predictions)
  metrics = precision_recall_fscore_support(integerLabels, predictions, average='weighted')
  confusion = confusion_matrix(integerLabels, predictions)

  return predictions, probabilities, accuracy, metrics, confusion

"""# Future Work"""

# Dataset Used - FERG-DB

# To do Tasks:
# 1) Implement K-fold cross validation
# 2) Cartoonize human images
# 3) Test Model on cartoonized human images
# 4) Enable live face tracking and cartoonization using openCV
# 5) Test effects of image augmentation
#       - Geographic Transformations
#       - Flipping sideways
#       - Colour Transformations
#       - Cropping measures
# 6) Track how model performs on human images at different levels of available data (10 images, 100 images, 1000 images, complete dataset, etc.)
# 7) Experiment with other bottleneck feature extractors than the VGG - Residual Networks, Inception Networks, DenseNets, etc.

# 17 - train with cropped faces with animated and then with human
# 18 - use basic cartoonization on human images and test accuracies
# 19, 20 - get final accuracies somehow and begin research paper
# 21, 22 - implement opencv functionality

# 25 - cartoonize cohn kanade dataset with cgan
# 26 - Write functions to implement kfold, get accuracies for different data percentages, different augmentation techniques(flip image), and number of epochs
# 27 - complete base research paper
# 28 - format final research paper
# 29 - opencv implementation
# 30 -

"""# Experimental"""

test_df = cartoon_test.sample(frac = 0.01)

test_df.shape

model_bottleneck, model_cartoon = get_models()

predictions, probabilities, accuracy, metrics, confusion = test_cartoon_accuracy(model_bottleneck, model_cartoon, test_df, display_image = False)

metrics

accuracy

human_paths, human_df = get_human_paths()

human_images, human_labels = get_human_images(human_df)
human_images_ct, human_labels_ct = get_human_cartoonized_images(human_ct_df)

human_train, human_test = train_test_split(human_df, stratify = human_df["Label"], test_size = 0.2)

human_images_train, human_labels_train = get_human_images(human_train)
human_images_test, human_labels_test = get_human_images(human_test)

model_cartoon = train_human(model_bottleneck, model_cartoon, human_images_train, human_labels_train, epochs = 10)

human_predictions, human_probabilities, human_accuracy, human_metrics, human_confusion = test_human_accuracy(model_bottleneck, model_cartoon, human_test)

human_accuracy

human_ct_paths, human_ct_df = get_human_cartoonized_paths()

im1 = cv2.imread(human_ct_paths[0])

im1.shape

np.asarray(ImageOps.grayscale(Image.fromarray(im1))).shape

human_ct_train, human_ct_test = train_test_split(human_ct_df, stratify = human_ct_df["Label"], test_size = 0.3)

human_images_ct_train, human_labels_ct_train = get_human_cartoonized_images(human_ct_train)
human_images_ct_test, human_labels_ct_test = get_human_cartoonized_images(human_ct_test)

model_cartoon = train_human(model_bottleneck, model_cartoon, human_images_ct_train, human_labels_ct_train, epochs = 50)

human_ct_predictions, human_ct_probabilities, human_ct_accuracy, human_ct_metrics, human_ct_confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, human_ct_test, display_image=False)

human_ct_accuracy

# get kfold splits
# loop over number of epochs
# train model on data
# track accuracy, precision, recall, f1 on cartoonized images

human_images_ct, human_labels_ct, human_labels_ct_integers = get_human_cartoonized_images(human_ct_df, integer_labels = True)

"""# Data Collection"""

human_ct_paths, human_ct_df = get_human_cartoonized_paths()

class Data:
  percentage = -1
  epoch = -1
  fold_count = -1
  test_predictions = -1
  test_probabilities = -1
  test_accuracy = -1
  test_metrics = -1
  test_confusion = -1
  train_predictions = -1
  train_probabilities = -1
  train_accuracy = -1
  train_metrics = -1
  train_confusion = -1

def train_kfold(human_ct_df, epochs, percentages):
  data_list = []
  for percentage in percentages:
    if (percentage!=100):
      trainer, tester = train_test_split(human_ct_df, stratify = human_ct_df["Label"], test_size = 1-percentage/100)
    else:
      trainer = human_ct_df
      tester = pd.DataFrame(columns = human_ct_df.columns)
    
    skfold = StratifiedKFold(n_splits = 10, shuffle = (percentage == 100))
    human_images_ct, human_labels_ct, human_labels_ct_integers = get_human_cartoonized_images(trainer, integer_labels = True)
    fold_count = 0

    for train_index, test_index in skfold.split(human_images_ct, human_labels_ct_integers):
      train_images, test_images = human_images_ct[train_index], human_images_ct[test_index]
      train_labels, test_labels = human_labels_ct[train_index], human_labels_ct[test_index]
      model_bottleneck, model_cartoon = get_models()

      for epoch in range(epochs):
        print("Training Percentage: " + str(percentage) + " | Fold: " + str(fold_count) + " | Epoch :" + str(epoch))
        data_obj = Data()
        model_cartoon = train_human(model_bottleneck, model_cartoon, train_images, train_labels, epochs = 1)
        test_predictions, test_probabilities, test_accuracy, test_metrics, test_confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, pd.concat([trainer.iloc[test_index], tester]), display_image = False)
        train_predictions, train_probabilities, train_accuracy, train_metrics, train_confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, trainer.iloc[train_index], display_image = False)
        data_obj.percentage = percentage
        data_obj.epoch = epoch
        data_obj.fold_count = fold_count
        data_obj.test_predictions = test_predictions
        data_obj.test_probabilities = test_probabilities
        data_obj.test_accuracy = test_accuracy
        data_obj.test_metrics = test_metrics
        data_obj.test_confusion = test_confusion
        data_obj.train_predictions = train_predictions
        data_obj.train_probabilities = train_probabilities
        data_obj.train_accuracy = train_accuracy
        data_obj.train_metrics = train_metrics
        data_obj.train_confusion = train_confusion

        data_list.append(data_obj)

        collected = gc.collect()

      fold_count+=1

  return data_list

def train_kfold_augmented(human_ct_df, epochs, percentages):
  data_list = []
  for percentage in percentages:
    if (percentage!=100):
      trainer, tester = train_test_split(human_ct_df, stratify = human_ct_df["Label"], test_size = 1-percentage/100)
      print("not 100")
    else:
      trainer = human_ct_df.sample(frac = 1).reset_index(drop = True)
      tester = pd.DataFrame(columns = human_ct_df.columns)
      print("100")
    print(percentage==100)
    skfold = StratifiedKFold(n_splits = 10, shuffle = (percentage == 100))

    fold_count = 0

    for train_index, test_index in skfold.split(trainer, trainer["Label"]):
      train_images, train_labels = get_human_cartoonized_images(trainer.iloc[train_index], augment = True)
      test_images, test_labels = get_human_cartoonized_images(trainer.iloc[test_index], augment = True)
      model_bottleneck, model_cartoon = get_models()

      for epoch in range(epochs):
        print("Training Percentage: " + str(percentage) + " | Fold: " + str(fold_count) + " | Epoch: " + str(epoch))
        data_obj = Data()
        model_cartoon = train_human(model_bottleneck, model_cartoon, train_images, train_labels, epochs = 1)
        test_predictions, test_probabilities, test_accuracy, test_metrics, test_confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, pd.concat([trainer.iloc[test_index], tester]), display_image = False, augment = True)
        train_predictions, train_probabilities, train_accuracy, train_metrics, train_confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, trainer.iloc[train_index], display_image = False, augment = True)
        
        data_obj.percentage = percentage
        data_obj.epoch = epoch
        data_obj.fold_count = fold_count
        data_obj.test_predictions = test_predictions
        data_obj.test_probabilities = test_probabilities
        data_obj.test_accuracy = test_accuracy
        data_obj.test_metrics = test_metrics
        data_obj.test_confusion = test_confusion
        data_obj.train_predictions = train_predictions
        data_obj.train_probabilities = train_probabilities
        data_obj.train_accuracy = train_accuracy
        data_obj.train_metrics = train_metrics
        data_obj.train_confusion = train_confusion

        print(test_accuracy)
        print(train_accuracy)

        data_list.append(data_obj)

        collected = gc.collect()

      fold_count+=1
      

  return data_list

# for i in [100]:
#   data_augmented_newer = train_kfold_augmented(human_ct_df, epochs = 10, percentages=[i])
  # file_name = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_augmented_new" + str(i) + ".pkl"
  # open_file = open(file_name, "wb")
  # pickle.dump(data_augmented, open_file)
  # open_file.close()

# file_name = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_augmented_newer.pkl"
# open_file = open(file_name, "wb")
# pickle.dump(data_augmented_newer, open_file)
# open_file.close()

# for i in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
#   data_augmented = train_kfold(human_ct_df, epochs = 10, percentages=[i])
#   file_name = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_unaugmented_new" + str(i) + ".pkl"
#   open_file = open(file_name, "wb")
#   pickle.dump(data_augmented, open_file)
#   open_file.close()

# file_name = "data_90_100.pkl"
# open_file = open(file_name, "wb")
# pickle.dump(data, open_file)
# open_file.close()
# files.download(file_name)

# data_unaugmented = []
# for i in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
#   file_name = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_unaugmented_new" + str(i) + ".pkl"
#   with open(file_name, 'rb') as f:
#     data_unaugmented += pickle.load(f)
#   open_file.close()

file_name1 = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_augmented_final.pkl"
file_name2 = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_unaugmented_final.pkl"
file_name3 = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_unaugmented_newer.pkl"
file_name4 = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_augmented_newer.pkl"

with open(file_name1, 'rb') as f:
  data_augmented = pickle.load(f)
with open(file_name2, 'rb') as f:
  data_unaugmented = pickle.load(f)
with open(file_name3, 'rb') as f:
  data_unaugmented_newer = pickle.load(f)
with open(file_name4, 'rb') as f:
  data_augmented_newer = pickle.load(f)

data_augmented[900:1000] = data_augmented_newer
data_unaugmented[900:1000] = data_unaugmented_newer

# 27 - # Add image augmentation stuff, get all data
# 28 - # Plot all graphs, re-write the abstract, intro and methodologies
# 28 - # Write the pre-processing, dataset used, etc. sections
# 28 - # Compile into google doc document
# 29 - # Make IEEE Format

# filename = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_augmented.pkl"
# with open(filename, 'rb') as f:
#     data_augmented = pickle.load(f)

# filename = "/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/data_unaugmented.pkl"
# with open(filename, 'rb') as f:
#     data_unaugmented = pickle.load(f)

list(data_augmented[0].__dict__.keys())

# Plots:
# For 10 epochs, 10-fold median training and testing accuracies for augmented and unaugmented for each percentage of data [total 10 plots]
# Make dataframe with fold median training, testing accuracies, together with different labels for augmented and unaugmented for each percentage sepearately for each epoch

columns_list = ["Epoch", "Augmented_Training_Accuracy", "Augmented_Testing_Accuracy", "Augmented_Testing_FScore", "Unaugmented_Training_Accuracy", "Unaugmented_Testing_Accuracy", "Unaugmented_Testing_FScore"]

def get_dataframe(df_percentage):
  data_df = pd.DataFrame(columns = columns_list)

  for epoch_index in range(10):
    training_accuracy_unaugmented = []
    testing_accuracy_unaugmented = []
    testing_fscore_unaugmented = []
    training_accuracy_augmented = []
    testing_accuracy_augmented = []
    testing_fscore_augmented = []

    for index in range(epoch_index, len(data_unaugmented), 10):
      if (data_unaugmented[index].percentage != df_percentage):
        continue

      training_accuracy_unaugmented.append(data_unaugmented[index].train_accuracy)
      testing_accuracy_unaugmented.append(data_unaugmented[index].test_accuracy)
      training_accuracy_augmented.append(data_augmented[index].train_accuracy)
      testing_accuracy_augmented.append(data_augmented[index].test_accuracy)
      testing_fscore_augmented.append(data_augmented[index].test_metrics[2])
      testing_fscore_unaugmented.append(data_unaugmented[index].test_metrics[2])
    
    training_accuracy_augmented = median(training_accuracy_augmented)
    training_accuracy_unaugmented = median(training_accuracy_unaugmented)
    testing_accuracy_augmented = median(testing_accuracy_augmented)
    testing_accuracy_unaugmented = median(testing_accuracy_unaugmented)
    testing_fscore_augmented = median(testing_fscore_augmented)
    testing_fscore_unaugmented = median(testing_fscore_unaugmented)

    df2 = {'Epoch': epoch_index, 
           'Augmented_Training_Accuracy': training_accuracy_augmented, 
           'Augmented_Testing_Accuracy': testing_accuracy_augmented, 
           'Unaugmented_Training_Accuracy': training_accuracy_unaugmented, 
           'Unaugmented_Testing_Accuracy': testing_accuracy_unaugmented,
           'Augmented_Testing_FScore': testing_fscore_augmented,
           'Unaugmented_Testing_FScore': testing_fscore_unaugmented}

    data_df = data_df.append(df2, ignore_index = True)

  return data_df

data_df_10 = get_dataframe(10)
data_df_20 = get_dataframe(20)
data_df_30 = get_dataframe(30)
data_df_40 = get_dataframe(40)
data_df_50 = get_dataframe(50)
data_df_60 = get_dataframe(60)
data_df_70 = get_dataframe(70)
data_df_80 = get_dataframe(80)
data_df_90 = get_dataframe(90)
data_df_100 = get_dataframe(100)

data_df_list = [data_df_10, data_df_20, data_df_30, data_df_40, data_df_50, data_df_60, data_df_70, data_df_80, data_df_90, data_df_100]

data_df_90

count = 10
for i in data_df_list:
  figure = i.plot(x ='Epoch', y=['Augmented_Testing_FScore', 'Augmented_Testing_Accuracy', 'Augmented_Training_Accuracy', 'Unaugmented_Testing_FScore', 'Unaugmented_Testing_Accuracy', 'Unaugmented_Training_Accuracy'], kind = 'line')
  # fig = figure.get_figure()
  # fig.savefig("/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/Plots/Plot_" + str(count) + '.jpg')
  count+=10

df2 = pd.DataFrame(columns = ['Percentage', 'Maximum Augmented Accuracy', 'Maximum Unaugmented Accuracy', 'Augmented Weighted FScore', 'Unaugmented Weighted FScore'])
for index, df in list(enumerate(data_df_list)):
  if (index == 9):
    break
  print(str((index+1)*10) + " Percentage:")
  print(str(df["Augmented_Testing_Accuracy"].iloc[-1]) + ", " + str(df["Unaugmented_Testing_Accuracy"].iloc[-1]))
  df2 = df2.append({
      'Percentage': (index+1)*10,
      'Maximum Augmented Accuracy': round(df["Augmented_Testing_Accuracy"].iloc[-1], 3),
      'Maximum Unaugmented Accuracy': round(df["Unaugmented_Testing_Accuracy"].iloc[-1], 3),
      'Augmented Weighted FScore': round(df['Augmented_Testing_FScore'].iloc[-1], 3),
      'Unaugmented Weighted FScore': round(df['Unaugmented_Testing_FScore'].iloc[-1], 3)
  }, ignore_index = True)

# df2.to_csv("/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/results.csv")

df2

# human_paths, human_df = get_human_paths()
# human_images, human_labels = get_human_images(human_df)
# human_images_ct, human_labels_ct = get_human_cartoonized_images(human_ct_df)





# for index, path in list(enumerate(paths)):
#   display(ImageOps.grayscale(Image.fromarray(cv2.imread(path))))

# file_name = "data_augmented.pkl"
# file_name2 = "data_unaugmented.pkl"
# open_file = open(file_name, "wb")
# pickle.dump(data_augmented, open_file)
# open_file.close()
# open_file = open(file_name2, "wb")
# pickle.dump(data_unaugmented, open_file)
# open_file.close()



# !git clone https://github.com/FilipAndersson245/cartoon-gan

# paths = glob.glob("/content/drive/MyDrive/Capstone/anime/animated2/Saved_Data/comparison/*.png")
# for index, path in list(enumerate(paths)):
#   im1 = np.asarray(ImageOps.grayscale(Image.fromarray(cv2.imread(path)[:, :256])))
#   im2 = np.asarray(ImageOps.grayscale(Image.fromarray(cv2.imread(path)[:, 256:])))
#   cv2.imwrite("/content/drive/MyDrive/Capstone/anime/animated2/Testing/cartoon-gan-master/datasets/human/human_" + str(index+1) + ".png", im1)
#   cv2.imwrite("/content/drive/MyDrive/Capstone/anime/animated2/Testing/cartoon-gan-master/datasets/cartoon/cartoon_" + str(index+1) + ".png", im2)

"""# Demo"""

from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


repo = "CartoonGan-tensorflow"
!git clone https://github.com/mnicnc404/{repo}.git
os.chdir(os.path.join(repo))

clear_output()

model_bottleneck, model_cartoon = get_models()
model_demo = load_model("/content/drive/MyDrive/Capstone/anime/animated2/demo_model.h5")
model_cartoon = load_model("/content/drive/MyDrive/Capstone/anime/animated2/predictor_model")
# model_cartoon = load_model("/content/drive/MyDrive/Capstone/anime/animated2/demo_model_combined.h5")
styles = "shinkai"
key = {0:"anger", 1:"joy", 2:"surprise", 3:"disgust", 4:"fear", 5:"sadness", 6:"neutral"}
clear_output()

def cartoonize_captured():
  print("PROCESSING....")

  old_stdout = sys.stdout # backup current stdout
  sys.stdout = open(os.devnull, "w")

  !python cartoonize.py \
      --styles {styles} \
      --batch_size 1 \
      --comparison_view horizontal
  
  sys.stdout = old_stdout
  clear_output()

def process_image():
  photo_col = np.asarray(Image.fromarray(np.asarray(Image.open('photo.jpg'))[:, 80:560]).resize((256, 256)))
  photo = cv2.resize(np.asarray(Image.open('photo.jpg'))[:, 80:560], (256, 256))
 
  new_photo = np.asarray(ImageOps.grayscale(PIL.Image.fromarray(photo)).convert("RGB"))
  new_photo = new_photo[:, :, 1].reshape(256, 256)
  imagePath = sys.argv[1]
 
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  faces = faceCascade.detectMultiScale(
          new_photo,
          scaleFactor=1.3,
          minNeighbors=3,
          minSize=(30, 30)
  )
 
  print("Found {} Faces!".format(len(faces)))
  for (x, y, w, h) in faces:
      rect = cv2.rectangle(photo, (x-10, y-10), (x+w+10, y+h+10), (0, 255, 0), 2)
      cv2.rectangle(photo_col, (x-10, y-10), (x+w+10, y+h+10), (0, 255, 0), 2)
 
  crop_photo = cv2.resize(np.asarray(Image.fromarray(photo[y:y+h, x:x+w])), (256, 256))
  crop_photo_col = np.asarray(Image.fromarray(photo_col[y:y+h, x:x+w]).resize((256, 256)))
  gray = np.asarray(ImageOps.grayscale(Image.fromarray(crop_photo)).convert("RGB"))
  gray_col = np.asarray(ImageOps.grayscale(Image.fromarray(crop_photo_col)).convert("RGB"))
  pred_photo = np.expand_dims(gray, axis=0)
  pred_photo_col = np.expand_dims(gray_col, axis=0)
 
  files1 = glob.glob("/content/CartoonGan-tensorflow/input_images/*")
  files2 = glob.glob("/content/CartoonGan-tensorflow/output_images/comparison/*")
 
  for f in files1:
      os.remove(f)
  for f in files2:
      os.remove(f)
  if (os.path.exists("/content/CartoonGan-tensorflow/output_images/shinkai")):
    files3 = glob.glob("/content/CartoonGan-tensorflow/output_images/shinkai/*")
    for f in files3:
      os.remove(f)
  
  Image.fromarray(pred_photo_col[0]).save("/content/CartoonGan-tensorflow/input_images/pred_photo_col.jpg")
  Image.fromarray(photo_col).save("/content/CartoonGan-tensorflow/input_images/photo_col.jpg")
  
  cv2.imwrite("/content/CartoonGan-tensorflow/input_images/pred_photo.jpg", pred_photo[0])
  cv2.imwrite("/content/CartoonGan-tensorflow/input_images/crop_photo.jpg", crop_photo)
 
  cartoonize_captured()
 
  cartoon_me = np.asarray(ImageOps.grayscale(Image.open('/content/CartoonGan-tensorflow/output_images/comparison/pred_photo.jpg')).convert("RGB"))[:, 256:]
  cartoon_me = np.expand_dims(cartoon_me, axis=0)
 
  pred = model_temp.predict_classes(model_bottleneck.predict(cartoon_me))
  prob = model_temp.predict(model_bottleneck.predict(cartoon_me))
 
  if (prob[0][3]>0.5):
    preder = 3
  else:
    temp = prob[0][3]
    prob[0][3] = 0
    preder = np.argmax(prob[0])
    prob[0][3] = temp
 
  # display(Image.fromarray(photo))
  display(Image.open('/content/CartoonGan-tensorflow/output_images/comparison/photo_col.jpg'))
  display(ImageOps.grayscale(Image.open('/content/CartoonGan-tensorflow/output_images/comparison/pred_photo.jpg')))
  print()
  print("Detected Emotion: " + str(key[preder]))
  print()
  print(prob[0])
  print()

def run_demo():
  from IPython.display import Image as imer
  try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
    
    display(imer(filename))
  except Exception as err:

    print(str(err))
  process_image()

run_demo()

model_bottleneck, model_cartoon = get_models()
human_paths, human_df = get_human_paths()
human_images, human_labels = get_human_images(human_df.sample(frac = 1))

model_bottleneck, model_human = get_models()
model_cartoon = train_human(model_bottleneck, model_cartoon, human_images, human_labels, epochs = 30)
model_cartoon = train_human(model_bottleneck, model_cartoon, human_images_cartoon, human_labels_cartoon, epochs = 4)

model_cartoon = load_model("/content/drive/MyDrive/Capstone/anime/animated2/predictor_model")

model_temp = load_model("/content/drive/MyDrive/Capstone/anime/animated2/predictor_model")

predictions, probabilities, accuracy, metrics, confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, human_df_cartoon, display_image = False)

accuracy

human_paths_cartoon, human_df_cartoon = get_human_cartoonized_paths()
human_images_cartoon, human_labels_cartoon = get_human_cartoonized_images(human_df_cartoon.sample(frac = 1))

model_cartoon = train_human(model_bottleneck, model_cartoon, human_images_cartoon, human_labels_cartoon, epochs = 4)

predictions, probabilities, accuracy, metrics, confusion = test_human_cartoonized_accuracy(model_bottleneck, model_cartoon, human_df_cartoon, display_image = False)

accuracy

model_demo = load_model("/content/drive/MyDrive/Capstone/anime/animated2/demo_model_combined.h5")

model_cartoon.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = 'accuracy')

def process_image():
  photo = cv2.resize(np.asarray(Image.open('photo.jpg'))[:, 80:560], (256, 256))
  new_photo = np.asarray(ImageOps.grayscale(PIL.Image.fromarray(photo)).convert("RGB"))
  new_photo = new_photo[:, :, 1].reshape(256, 256)
  imagePath = sys.argv[1]

  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  faces = faceCascade.detectMultiScale(
          photo,
          scaleFactor=1.3,
          minNeighbors=3,
          minSize=(30, 30)
  )

  print("Found {} Faces!".format(len(faces)))
  for (x, y, w, h) in faces:
      rect = cv2.rectangle(photo, (x-10, y-10), (x+w+10, y+h+10), (0, 255, 0), 2)

  crop_photo = cv2.resize(np.asarray(Image.fromarray(photo[y:y+h, x:x+w])), (256, 256))
  gray = np.asarray(ImageOps.grayscale(Image.fromarray(crop_photo)).convert("RGB"))
  pred_photo = np.expand_dims(gray, axis=0)

  files1 = glob.glob("/content/CartoonGan-tensorflow/input_images/*")
  files2 = glob.glob("/content/CartoonGan-tensorflow/output_images/comparison/*")

  for f in files1:
      os.remove(f)
  for f in files2:
      os.remove(f)
  if (os.path.exists("/content/CartoonGan-tensorflow/output_images/shinkai")):
    files3 = glob.glob("/content/CartoonGan-tensorflow/output_images/shinkai/*")
    for f in files3:
      os.remove(f)
      
  cv2.imwrite("/content/CartoonGan-tensorflow/input_images/pred_photo.jpg", pred_photo[0])

  cartoonize_captured()

  cartoon_me = np.asarray(ImageOps.grayscale(Image.open('/content/CartoonGan-tensorflow/output_images/comparison/pred_photo.jpg')).convert("RGB"))[:, 256:]
  cartoon_me = np.expand_dims(cartoon_me, axis=0)

  pred = model_cartoon.predict_classes(model_bottleneck.predict(cartoon_me))
  prob = model_cartoon.predict(model_bottleneck.predict(cartoon_me))

  if (prob[0][3]>0.75):
    preder = 3
  else:
    temp = prob[0][3]
    prob[0][3] = 0
    preder = np.argmax(prob[0])
    prob[0][3] = temp

  display(ImageOps.grayscale(Image.open('/content/CartoonGan-tensorflow/output_images/comparison/pred_photo.jpg')))
  print()
  print("Detected Emotion: " + str(key[preder]))
  print()
  print(prob[0])
  print()

"""# Experimental 2"""

temp = glob.glob("/content/drive/MyDrive/Capstone/TD_CS_Set1/*/TD_RGB*")

lister = [Image.open(i) for i in temp]

files1 = glob.glob("/content/CartoonGan-tensorflow/input_images/*")
files2 = glob.glob("/content/CartoonGan-tensorflow/output_images/comparison/*")

for f in files1:
    os.remove(f)
for f in files2:
    os.remove(f)
if (os.path.exists("/content/CartoonGan-tensorflow/output_images/shinkai")):
  files3 = glob.glob("/content/CartoonGan-tensorflow/output_images/shinkai/*")
  for f in files3:
    os.remove(f)

count = 0
for i in lister:
  i.save("/content/CartoonGan-tensorflow/input_images/" + str(count) + ".jpg", 'JPEG')
  count+=1

cartoonize_captured()

new_glob = glob.glob("/content/CartoonGan-tensorflow/output_images/comparison/*")

new_list = [Image.open(i) for i in new_glob]

conc1 = np.asarray(new_list[0])
conc2 = np.asarray(new_list[5])
conc3 = np.asarray(new_list[10])
conc4 = np.asarray(new_list[15])

for i in new_list[1:5]:
  conc1 = np.hstack([conc1, i])
for i in new_list[6:10]:
  conc2 = np.hstack([conc2, i])
for i in new_list[11:15]:
  conc3 = np.hstack([conc3, i])
for i in new_list[16:20]:
  conc4 = np.hstack([conc4, i])
final = np.vstack([conc1, conc2, conc3, conc4])

display(Image.fromarray(final))